## 1. Setup and Installation

Install required packages:

In [ ]:
%pip install --upgrade pip
%pip install numpy --upgrade --only-binary :all:
%pip install adversarial-robustness-toolbox torch torchvision pillow matplotlib opencv-python --only-binary :all:

## 2. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import cv2
import os
import json
import urllib.request

from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from art.estimators.classification import PyTorchClassifier

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 3. Load Pre-trained Model and Labels

We'll use MobileNetV2 trained on ImageNet, which includes many automotive classes (cars, trucks, buses, etc.)

In [ ]:
# Load MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.eval()

# Download ImageNet labels
IMAGENET_LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
with urllib.request.urlopen(IMAGENET_LABELS_URL) as url:
    imagenet_labels = json.loads(url.read().decode())

print("Model loaded successfully!")
print(f"Model type: {type(model).__name__}")

## 4. Load Real Automotive Images

Download and prepare real automotive images for testing

In [ ]:
# URLs for sample automotive images
image_urls = [
    "https://images.unsplash.com/photo-1549317661-bd32c8ce0db2?w=800&q=80",  # Sports car
    "https://images.unsplash.com/photo-1583121274602-3e2820c69888?w=800&q=80",  # SUV
    "https://images.unsplash.com/photo-1552519507-da3b142c6e3d?w=800&q=80",  # Sedan
]

def download_and_prepare_image(url, max_size=(224, 224)):
    """Download image from URL and resize it"""
    try:
        with urllib.request.urlopen(url) as response:
            img_array = np.asarray(bytearray(response.read()), dtype=np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Resize while maintaining aspect ratio
            h, w = img_rgb.shape[:2]
            scale = min(max_size[0]/h, max_size[1]/w)
            new_h, new_w = int(h*scale), int(w*scale)
            img_resized = cv2.resize(img_rgb, (new_w, new_h))
            return img_resized
    except Exception as e:
        print(f"Error loading image from {url}: {e}")
        return None

# Download all images
print("Downloading automotive images...")
sample_images = []
for i, url in enumerate(image_urls):
    img = download_and_prepare_image(url)
    if img is not None:
        sample_images.append(img)
        print(f"✓ Image {i+1} downloaded successfully")

print(f"\nTotal images loaded: {len(sample_images)}")

# Display all images
fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
if len(sample_images) == 1:
    axes = [axes]

for idx, (img, ax) in enumerate(zip(sample_images, axes)):
    ax.imshow(img)
    ax.set_title(f"Automotive Image {idx+1}")
    ax.axis('off')

plt.tight_layout()
plt.show()

## 5. Preprocess Images and Get Predictions

Preprocess all images and get model predictions

In [ ]:
def load_and_preprocess_image(img_array):
    """Preprocess image for PyTorch MobileNetV2"""
    # Convert BGR to RGB if needed
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) if len(img_array.shape) == 3 else img_array
    img_pil = Image.fromarray(img_rgb)
    
    # PyTorch preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    img_tensor = preprocess(img_pil)
    return img_tensor.unsqueeze(0).numpy()

def get_predictions(model, img_tensor, top_k=3):
    """Get top predictions from PyTorch model"""
    with torch.no_grad():
        img_torch = torch.from_numpy(img_tensor)
        outputs = model(img_torch)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
    
    top_prob, top_idx = torch.topk(probabilities, top_k)
    results = [(idx.item(), imagenet_labels[idx.item()], prob.item()) 
               for idx, prob in zip(top_idx, top_prob)]
    return results

# Preprocess all images and get predictions
preprocessed_images = []
all_predictions = []

print("Processing images and getting predictions...\n")
for idx, img in enumerate(sample_images):
    x_test = load_and_preprocess_image(img)
    preprocessed_images.append(x_test)
    
    decoded_preds = get_predictions(model, x_test, top_k=3)
    all_predictions.append(decoded_preds)
    
    print(f"Image {idx+1} - Original Predictions:")
    for i, (label_idx, label, score) in enumerate(decoded_preds):
        print(f"  {i+1}. {label}: {score*100:.2f}%")
    print()

## 6. Wrap Model for ART

Wrap the TensorFlow model with ART's classifier wrapper

In [ ]:
# Define loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Wrap the model with ART
classifier = PyTorchClassifier(
    model=model,
    loss=loss_fn,
    input_shape=(3, 224, 224),
    nb_classes=1000,
    clip_values=(-3, 3)  # Normalized range for PyTorch
)

print("Model wrapped successfully for ART!")

## 7. Attack 1: Fast Gradient Sign Method (FGSM)

FGSM is a simple and fast attack that perturbs the image in the direction of the gradient.

In [ ]:
# Create FGSM attack
fgsm_attack = FastGradientMethod(estimator=classifier, eps=0.3)

# Generate adversarial examples for all images
print("Generating FGSM adversarial examples...\n")
fgsm_adversarial_images = []
fgsm_predictions = []

for idx, x_test in enumerate(preprocessed_images):
    # Generate adversarial example
    x_test_adv = fgsm_attack.generate(x=x_test)
    fgsm_adversarial_images.append(x_test_adv)
    
    # Get predictions on adversarial image
    decoded_preds_adv = get_predictions(model, x_test_adv, top_k=3)
    fgsm_predictions.append(decoded_preds_adv)
    
    print(f"Image {idx+1} - FGSM Adversarial Predictions:")
    for i, (label_idx, label, score) in enumerate(decoded_preds_adv):
        print(f"  {i+1}. {label}: {score*100:.2f}%")
    print()

### Visualize FGSM Attack Results

In [ ]:
def denormalize_pytorch(img_tensor):
    """Denormalize PyTorch tensor for display"""
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = img_tensor[0].transpose(1, 2, 0)
    img = std * img + mean
    img = np.clip(img * 255, 0, 255).astype(np.uint8)
    return img

# Visualize FGSM results for all images
fig, axes = plt.subplots(len(sample_images), 3, figsize=(15, 5*len(sample_images)))
if len(sample_images) == 1:
    axes = axes.reshape(1, -1)

for idx in range(len(sample_images)):
    # Original image
    axes[idx, 0].imshow(sample_images[idx])
    orig_pred = all_predictions[idx][0]
    axes[idx, 0].set_title(f"Original Image {idx+1}\nTop: {orig_pred[1]}\n({orig_pred[2]*100:.1f}%)")
    axes[idx, 0].axis('off')
    
    # Adversarial image
    adv_img_display = denormalize_pytorch(fgsm_adversarial_images[idx])
    axes[idx, 1].imshow(adv_img_display)
    adv_pred = fgsm_predictions[idx][0]
    axes[idx, 1].set_title(f"FGSM Attack (eps=0.3)\nTop: {adv_pred[1]}\n({adv_pred[2]*100:.1f}%)")
    axes[idx, 1].axis('off')
    
    # Perturbation (amplified for visibility)
    perturbation = (fgsm_adversarial_images[idx][0] - preprocessed_images[idx][0]).transpose(1, 2, 0) * 50
    perturbation = np.clip(perturbation + 128, 0, 255).astype(np.uint8)
    axes[idx, 2].imshow(perturbation)
    axes[idx, 2].set_title("Perturbation\n(50x amplified)")
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.show()

## 8. Attack 2: Projected Gradient Descent (PGD)

PGD is an iterative attack that is more powerful than FGSM.

In [ ]:
# Create PGD attack
pgd_attack = ProjectedGradientDescent(
    estimator=classifier,
    eps=0.3,
    eps_step=0.03,
    max_iter=20,
    targeted=False
)

# Generate adversarial examples for all images
print("Generating PGD adversarial examples...\n")
pgd_adversarial_images = []
pgd_predictions = []

for idx, x_test in enumerate(preprocessed_images):
    # Generate adversarial example
    x_test_adv = pgd_attack.generate(x=x_test)
    pgd_adversarial_images.append(x_test_adv)
    
    # Get predictions on adversarial image
    decoded_preds_adv = get_predictions(model, x_test_adv, top_k=3)
    pgd_predictions.append(decoded_preds_adv)
    
    print(f"Image {idx+1} - PGD Adversarial Predictions:")
    for i, (label_idx, label, score) in enumerate(decoded_preds_adv):
        print(f"  {i+1}. {label}: {score*100:.2f}%")
    print()

### Visualize PGD Attack Results

In [ ]:
# Visualize PGD results for all images
fig, axes = plt.subplots(len(sample_images), 3, figsize=(15, 5*len(sample_images)))
if len(sample_images) == 1:
    axes = axes.reshape(1, -1)

for idx in range(len(sample_images)):
    # Original image
    axes[idx, 0].imshow(sample_images[idx])
    orig_pred = all_predictions[idx][0]
    axes[idx, 0].set_title(f"Original Image {idx+1}\nTop: {orig_pred[1]}\n({orig_pred[2]*100:.1f}%)")
    axes[idx, 0].axis('off')
    
    # Adversarial image
    adv_img_display = denormalize_pytorch(pgd_adversarial_images[idx])
    axes[idx, 1].imshow(adv_img_display)
    adv_pred = pgd_predictions[idx][0]
    axes[idx, 1].set_title(f"PGD Attack (eps=0.3)\nTop: {adv_pred[1]}\n({adv_pred[2]*100:.1f}%)")
    axes[idx, 1].axis('off')
    
    # Perturbation (amplified for visibility)
    perturbation = (pgd_adversarial_images[idx][0] - preprocessed_images[idx][0]).transpose(1, 2, 0) * 50
    perturbation = np.clip(perturbation + 128, 0, 255).astype(np.uint8)
    axes[idx, 2].imshow(perturbation)
    axes[idx, 2].set_title("Perturbation\n(50x amplified)")
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.show()

## 9. Attack 3: Carlini & Wagner L2 Attack

C&W attack is a powerful optimization-based attack.

In [ ]:
# Create C&W attack (using fewer iterations for speed)
cw_attack = CarliniL2Method(
    classifier=classifier,
    confidence=0.0,
    targeted=False,
    max_iter=10,  # Reduced for speed
    learning_rate=0.01
)

# Generate adversarial examples for all images
print("Generating C&W adversarial examples (this may take a moment)...\n")
cw_adversarial_images = []
cw_predictions = []

for idx, x_test in enumerate(preprocessed_images):
    print(f"Processing Image {idx+1}...")
    # Generate adversarial example
    x_test_adv = cw_attack.generate(x=x_test)
    cw_adversarial_images.append(x_test_adv)
    
    # Get predictions on adversarial image
    decoded_preds_adv = get_predictions(model, x_test_adv, top_k=3)
    cw_predictions.append(decoded_preds_adv)
    
    print(f"Image {idx+1} - C&W Adversarial Predictions:")
    for i, (label_idx, label, score) in enumerate(decoded_preds_adv):
        print(f"  {i+1}. {label}: {score*100:.2f}%")
    print()

### Visualize C&W Attack Results

In [ ]:
# Visualize C&W results for all images
fig, axes = plt.subplots(len(sample_images), 3, figsize=(15, 5*len(sample_images)))
if len(sample_images) == 1:
    axes = axes.reshape(1, -1)

for idx in range(len(sample_images)):
    # Original image
    axes[idx, 0].imshow(sample_images[idx])
    orig_pred = all_predictions[idx][0]
    axes[idx, 0].set_title(f"Original Image {idx+1}\nTop: {orig_pred[1]}\n({orig_pred[2]*100:.1f}%)")
    axes[idx, 0].axis('off')
    
    # Adversarial image
    adv_img_display = denormalize_pytorch(cw_adversarial_images[idx])
    axes[idx, 1].imshow(adv_img_display)
    adv_pred = cw_predictions[idx][0]
    axes[idx, 1].set_title(f"C&W Attack\nTop: {adv_pred[1]}\n({adv_pred[2]*100:.1f}%)")
    axes[idx, 1].axis('off')
    
    # Perturbation (amplified for visibility)
    perturbation = (cw_adversarial_images[idx][0] - preprocessed_images[idx][0]).transpose(1, 2, 0) * 50
    perturbation = np.clip(perturbation + 128, 0, 255).astype(np.uint8)
    axes[idx, 2].imshow(perturbation)
    axes[idx, 2].set_title("Perturbation\n(50x amplified)")
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.show()

## 10. Compare All Attacks Side-by-Side

In [ ]:
# Compare all attacks side-by-side for each image
for img_idx in range(len(sample_images)):
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    
    # Row 1: Images
    # Original
    axes[0, 0].imshow(sample_images[img_idx])
    axes[0, 0].set_title(f"Original Image {img_idx+1}", fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    # FGSM
    fgsm_display = denormalize_pytorch(fgsm_adversarial_images[img_idx])
    axes[0, 1].imshow(fgsm_display)
    axes[0, 1].set_title("FGSM Attack", fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    
    # PGD
    pgd_display = denormalize_pytorch(pgd_adversarial_images[img_idx])
    axes[0, 2].imshow(pgd_display)
    axes[0, 2].set_title("PGD Attack", fontsize=12, fontweight='bold')
    axes[0, 2].axis('off')
    
    # C&W
    cw_display = denormalize_pytorch(cw_adversarial_images[img_idx])
    axes[0, 3].imshow(cw_display)
    axes[0, 3].set_title("C&W Attack", fontsize=12, fontweight='bold')
    axes[0, 3].axis('off')
    
    # Row 2: Predictions
    predictions_data = [
        ("Original", all_predictions[img_idx]),
        ("FGSM", fgsm_predictions[img_idx]),
        ("PGD", pgd_predictions[img_idx]),
        ("C&W", cw_predictions[img_idx])
    ]
    
    for idx, (title, preds) in enumerate(predictions_data):
        text = "Top 3 Predictions:\n\n"
        for i, (class_idx, label, score) in enumerate(preds):
            text += f"{i+1}. {label}\n   {score*100:.1f}%\n\n"
        
        axes[1, idx].text(0.1, 0.9, text, fontsize=10, verticalalignment='top',
                         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
        axes[1, idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 11. Summary

### Key Observations:

1. **FGSM (Fast Gradient Sign Method)**:
   - Fastest attack
   - Single-step gradient-based
   - Creates visible but effective perturbations

2. **PGD (Projected Gradient Descent)**:
   - Iterative version of FGSM
   - More powerful than FGSM
   - Better at fooling the model

3. **C&W (Carlini & Wagner)**:
   - Optimization-based attack
   - Can create minimal perturbations
   - Slower but very effective

### Automotive AI Implications:

- These attacks demonstrate vulnerabilities in image classification models
- Critical for autonomous driving systems that rely on visual perception
- Small perturbations can cause misclassification of vehicles, signs, or pedestrians
- Robustness testing is essential for safety-critical automotive AI systems